In [ ]:
import ase
from ase import Atoms
from ase.visualize import view
from ase.build import bulk, stack
from ase.spacegroup import crystal
from ase.build.tools import niggli_reduce, cut, niggli_reduce_cell
from ase.spacegroup import crystal
from pymatgen.io.cif import CifParser
import numpy as np
import pymatgen as mg
from pymatgen.io.cif import CifParser
from sklearn.neighbors import KDTree
import numpy as np
import csv
import os
import sklearn as skl
from sklearn.neighbors import KDTree

In [ ]:
#file = "SrTiO3_Pm-3m_80874_80874.cif"
file = "icsd_000005.cif"
structures = CifParser(file).get_structures()[0]
lattice = structures.lattice
formula = structures.formula.replace(" ", "")
scaled_positions = structures.frac_coords
angles = structures.lattice.angles
abc = structures.lattice.abc
a,b,c = structures.lattice.abc
cell = abc+angles
atom = Atoms(formula, cell=cell, pbc=True, scaled_positions=scaled_positions)
reduced, op = niggli_reduce_cell(np.array(atom.get_cell()))
reduced_atom = Atoms(formula, cell=reduced, pbc=True, scaled_positions=scaled_positions)

In [ ]:
def divide(l, n): 
      
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n]  

# One file

In [ ]:
def Supercell(atom,e):
    P = [[e,0,0],[0,e,0],[0,0,e]]
    supercell = ase.build.make_supercell(atom,P)
    return supercell

In [ ]:
def Tree(supercell):
    x = list(divide(supercell.positions, n)) 
    mp = int(np.ceil(len(x)/2))
    tree = KDTree(supercell.positions,metric="euclidean")
    Ns_1, ind = tree.query(x[mp-1], k=k)
    return Ns_1

# One file

In [ ]:
k = int(input('Input k:'))
n = len(reduced_atom.positions)
m = len(structures.sites)
e = np.ceil(np.cbrt(k/m)) #highest integer
if e % 2 == 0:
    e = e + 1

s = 0
Ns = np.ones((m,k))
Ns_1 = Tree(Supercell(reduced_atom,e))

while (Ns == Ns_1).all() == False:
    
    #print(Ns)
    #print(Ns_1)
    
    Ns = Ns_1
    e = e + 2
    
    #Ns_1 = Tree(Supercell(e))
    Ns_1 = np.around(Tree(Supercell(reduced_atom,e)), decimals=10)
    
    #print(Ns)
    #print(Ns_1)
    
    s = s + 1
    
    print('No match at step',s)
    
print('Done at step',s+1)
AMD_vector = []
for i in range(k):
    avg = np.mean(Ns_1[:,i])
    AMD_vector.append(avg)

print(AMD_vector)
#print(Ns_1)

# All files in directory

In [ ]:
directory = "/Users/admin/ICSD_2019_Li_CIFs"

In [ ]:
files = []
for file in os.listdir(directory):
    if file.endswith(".cif"):
        new_file = os.path.join(directory, file)
        files.append(os.path.join(directory, new_file))

In [ ]:
file_array = np.array(files)
#file_array

In [ ]:
reduced_cells = []
ops = []
atoms = []
sites = []

for file in file_array:
    structures = CifParser(file).get_structures()[0]
    formula = structures.formula.replace(" ", "")
    scaled_positions = structures.frac_coords
    angles = structures.lattice.angles
    abc = structures.lattice.abc
    cell = abc+angles
    atom = Atoms(formula, cell=cell, pbc=True, scaled_positions=scaled_positions)
    reduced, op = niggli_reduce_cell(np.array(atom.get_cell()))
    reduced_cells.append(reduced)
    ops.append(op)

    sites.append(structures.sites)
    atoms.append(Atoms(formula, cell=reduced, pbc=True, scaled_positions=scaled_positions))

In [ ]:
k = int(input('Input k:'))

    
for i in range(len(atoms)):
    
    n = len(atoms[i].positions)
    m = len(sites[i])
    e = np.ceil(np.cbrt(k/m)) #highest integer
    if e % 2 == 0:
        e = e + 1

    s = 0
    Ns = np.ones((m,k))
    Ns_1 = Tree(Supercell(atoms[i],e))
    
    while (Ns == Ns_1).all() == False:

        Ns = Ns_1
        e = e + 2
        
        Ns_1 = np.around(Tree(Supercell(atoms[i],e)), decimals=10)

        s = s + 1

        print('Failed at step',s)

    print('Done at step',s+1)
    AMD_vector = []
    for i in range(k):
        avg = np.mean(Ns_1[:,i])
        AMD_vector.append(avg)

    print(AMD_vector)